## Shared Imports

In [128]:
from dataHandler.importer import updateDB
from config import NETWORK, USERNAME, DBFILE

import ipywidgets as widgets

import matplotlib.pyplot as plt

## Temporary Section

In [129]:
import pandas as pd
from dataHandler.models import Base, Scrobble
import datetime as dt

__create db session__

In [130]:
# this section should be made into a decorator
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
engine = create_engine(('sqlite:///' + DBFILE), echo=False)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

__sql db to pandas df__ 

In [131]:
query = 'SELECT C.id, C.timestamp, C.track, C.album, C.artist, (SELECT COUNT(O.track) + 1 FROM [scrobbles] O WHERE O.track = C.track and O.artist = C.artist and O.timestamp < C.timestamp) as trackCount FROM scrobbles C ORDER BY C.timestamp'

In [132]:
# query = session.query(Scrobble)
records_df = pd.read_sql(query, session.bind)

In [134]:
records_df.loc[records_df['track'] == 'Say Something']

,id,timestamp,track,album,artist,trackCount
1482,11084,1493595075,Say Something,Covers Part III,Anthem Lights,1
1950,10616,1496028684,Say Something,As Seen On TV,Alex & Sierra,1
2790,9776,1500937694,Say Something,Is There Anybody Out There?,A Great Big World,1
2798,9768,1501017272,Say Something,Is There Anybody Out There?,A Great Big World,2
7590,4976,1525400805,Say Something,As Seen On TV,Alex & Sierra,2
15866,15724,1553826968,Say Something,Is There Anybody Out There?,A Great Big World,3


In [135]:
session.close()